In [1]:
!pip install -U efficientnet

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from API import *

#!pip install tensor-dash
#from tensordash.tensordash import Tensordash

In [2]:
CFG = get_config()

Using default strategy for CPU and single GPU
Num GPUs Available:  1
REPLICAS: 1


# Balancing / Reducing dataset :
1/6 = malignant and 5/6 = benign

In [ ]:
# df = pd.read_csv('data/labels.csv')
# #cleaning data
# df.image_name = df.image_name + ".png"
# df['target'] = df['benign_malignant']
# df = df[['image_name', 'target']]
# df.columns = ['filename', 'target']
# #Reducing dataset size: 1/6 = malignant and 5/6 = benign
# malignants = df[df.target=='malignant']
# nbMalignant = len(malignants)
# benign = df[df.target=='benign']
# benign2 = benign.sample(nbMalignant*5, replace=False)
# #concatenating both parts + shuffle
# df = pd.concat([malignants,benign2])
# df = newdf.sample(frac=1).reset_index(drop=True)

# df.to_csv("data/df_clean.csv", index=False)
df = pd.read_csv("data/df_clean.csv")
df.head()

In [ ]:
# files = os.listdir("data/tb3images")

# dftmp = []
# for i in range(len(files)):
#     if len(files[i].split('_'))==2:
#         dftmp.append(files[i])
# dftmp = pd.DataFrame(dftmp)
# dftmp.columns=['filename']

# df = pd.read_csv('data/tb3labels.csv')
# df.image_id = df.image_id + ".jpg"
# df = df[['image_id', 'melanoma']]
# df.columns = ['filename', 'target']
# tmp = df.copy()
# tmp[tmp.target==0] = 'benign'
# tmp[tmp.target==1] = 'malignant'
# df['target'] = tmp['target']

# idx = [np.array(df['filename'])[i] in np.array(dftmp['filename']) for i in range(len(df))]
# df = df[idx]
# df

# Create Segmentations
Possibility to skip this step if we give a segmentation to the input 

In [ ]:
#get_segmentations(CFG, df, "data/images/", "data/masks/")

# Create Dataframe with tabular features

In [ ]:
# df = get_tabular_dataframe(df, "data/images/", "data/masks/")
# df.to_csv("data/df_tabular.csv", index=False)
df = pd.read_csv("data/df_tabular.csv")
df.head()

# Create TFRecord

In [ ]:
#write_tfrecord(CFG, df, "data/images/", "data/tfrecord_labeled.tfrec", True)

# Read TFRecord

In [ ]:
#dataset_train = read_tfrecord(CFG, tfrec, augment=True, repeat=True, shuffle=True)
#this dataset will be used for evaluating, this is why we don't augment/shuffle/repeat
#Normally, here, we have only 1 img in this dataset
#dataset_test = read_tfrecord(CFG, tfrec, augment=False, repeat=False, shuffle=False, ordered=True)

dataset = read_tfrecord(CFG, "data/tfrecord_labeled.tfrec", augment=False, repeat=False, shuffle=False, ordered=True, labeled=True)

# Create Model

In [ ]:
m = get_model(CFG, fine_tune=False, model_weights="models/model_effnetB0-4_noisy-student_transfer-learning.h5")
#m.summary()

## fit the model

In [ ]:
# callbacks = [tf.keras.callbacks.ReduceLROnPlateau(),
#              tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
#              tf.keras.callbacks.ModelCheckpoint("models/best.h5", save_best_only=True, monitor='val_auc', mode='max', save_weights_only=True)]

# stepstrain = len(df)//CFG['batch_size'] 
# if len(df)%CFG['batch_size'] > 0: stepstrain+=1

# m.fit(
#     dataset_train,
#     steps_per_epoch=stepstrain,
#     epochs=CFG['epochs'],
#     callbacks=callbacks,
# )

## predict

In [ ]:
preds = m.predict(
        dataset,
        steps = len(df)/CFG['batch_size'],
)

In [ ]:
ytrue = np.array(df['target']=="malignant").astype('int64')
ypreds = np.argmax(preds, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay

#Nos predictions en colonne
cm = confusion_matrix(ytrue, ypreds)
disp = ConfusionMatrixDisplay(cm, display_labels=["benign", "malignant"])
disp.plot(cmap=plt.cm.Blues)
disp.ax_.set_title("Confusion Matrix, threshold: 0.5")

accuracy = np.mean(ytrue==ypreds)
precision = cm[0,0]/cm.sum(axis=0)[0]
recall = cm[0,0]/cm.sum(axis=1)[0]
F1score = 2*(recall*precision)/(recall+precision)
print("accuracy:",accuracy)
print("precision:",precision)
print("recall:",recall)
print("F1score:",F1score)

In [ ]:
ismalignant = 0.70
ypreds2 = (preds[:,1]>ismalignant).astype('int')

cm = confusion_matrix(ytrue, ypreds2)
disp = ConfusionMatrixDisplay(cm, display_labels=["benign", "malignant"])
disp.plot(cmap=plt.cm.Blues)
disp.ax_.set_title("Confusion Matrix, threshold: 0.35")

accuracy = np.mean(ytrue==ypreds2)
precision = cm[0,0]/cm.sum(axis=0)[0]
recall = cm[0,0]/cm.sum(axis=1)[0]
F1score = 2*(recall*precision)/(recall+precision)
print("accuracy:",accuracy)
print("precision:",precision)
print("recall:",recall)
print("F1score:",F1score)